In [1]:
import pandas as pd

# Import data and skip the first 3 rows
data = pd.read_csv('./excel.csv', skiprows=3)

# Cleaning column names
data.columns = data.columns.str.strip()

# Force non-numeric values to NaN ignoring the comma seperation in large numbers
data[data.columns[0]] = data[data.columns[0]].apply(pd.to_numeric, errors='coerce')
data[data.columns[2:]] = data[data.columns[2:]].replace({',': ''}, regex=True).apply(pd.to_numeric, errors='coerce')

# Delete rows not having serial number
data.dropna(subset=[data.columns[0]], inplace=True)

# Drop the first column since we don't need it anymore
data.drop(columns=[data.columns[0]], inplace=True)

data.head(5)


,Country,2008-2009,%Share,2009-2010,%Share .1,2010-2011,%Share .2,2011-2012,%Share .3,2012-2013,...,2020-2021,%Share .12,2021-2022,%Share .13,2022-2023,%Share .14,2023-2024,%Share .15,2024-2025,%Share .16
0,AFGHANISTAN,182344.16,0.0,220362.72,0.0,192084.0,0.0,242912.0,0.0,256914.0,...,610620.0,0.0,412904.0,0.0,349498.0,0.0,294271.0,0.0,52979.0,0.0
1,ALBANIA,5589.70,0.0,4027.54,0.0,5227.0,0.0,6067.0,0.0,9516.0,...,34619.0,0.0,41040.0,0.0,282091.0,0.0,955574.0,0.0,10653.0,0.0
2,ALGERIA,299636.27,0.0,273710.63,0.0,355787.0,0.0,399230.0,0.0,593470.0,...,439810.0,0.0,523801.0,0.0,493477.0,0.0,702538.0,0.0,119886.0,0.0
3,AMERI SAMOA,58.26,0.0,40.45,NaN,93.0,0.0,975.0,0.0,199.0,...,433.0,0.0,578.0,0.0,23.0,NaN,3440.0,0.0,814.0,0.0
4,ANDORRA,412.37,0.0,96.49,0.0,120.0,0.0,125.0,0.0,59.0,...,15.0,NaN,37.0,NaN,181.0,0.0,262.0,0.0,NaN,NaN


In [2]:
# Calculate the correlation between shares and amounts

for i in range(1, len(data.columns) - 1, 2):
    
    correlation = data[data.columns[i: i+2]].corr()
    print(correlation)


           2008-2009    %Share
2008-2009   1.000000  0.992457
%Share      0.992457  1.000000
           2009-2010  %Share .1
2009-2010   1.000000   0.994126
%Share .1   0.994126   1.000000
           2010-2011  %Share .2
2010-2011   1.000000   0.995185
%Share .2   0.995185   1.000000
           2011-2012  %Share .3
2011-2012   1.000000   0.993442
%Share .3   0.993442   1.000000
           2012-2013  %Share .4
2012-2013    1.00000    0.99365
%Share .4    0.99365    1.00000
           2013-2014  %Share .5
2013-2014   1.000000   0.992477
%Share .5   0.992477   1.000000
           2014-2015  %Share .6
2014-2015   1.000000   0.994192
%Share .6   0.994192   1.000000
           2015-2016  %Share .7
2015-2016   1.000000   0.993732
%Share .7   0.993732   1.000000
           2016-2017  %Share .8
2016-2017   1.000000   0.994001
%Share .8   0.994001   1.000000
           2017-2018  %Share .9
2017-2018   1.000000   0.993678
%Share .9   0.993678   1.000000
            2018-2019  %Share .10
2018-2019

### Attention

As we saw in the previous cell, the correlation between '%Shares' columns and their relevant year is almost <b>1</b>. So it's not necessary to include those columns in our analysis.

In [3]:
# Drop Shares columns
data.drop(columns=[col for col in data.columns if '%Share' in col], inplace=True)

# Rename columns
data.rename(columns={col: 'Year_' + col.replace('-', '_') for col in data.columns[1:]}, inplace=True)

data.head(2)


,Country,Year_2008_2009,Year_2009_2010,Year_2010_2011,Year_2011_2012,Year_2012_2013,Year_2013_2014,Year_2014_2015,Year_2015_2016,Year_2016_2017,Year_2017_2018,Year_2018_2019,Year_2019_2020,Year_2020_2021,Year_2021_2022,Year_2022_2023,Year_2023_2024,Year_2024_2025
0,AFGHANISTAN,182344.16,220362.72,192084.0,242912.0,256914.0,287888.0,258130.0,343603.0,339682.0,457689.0,499237.0,708458.0,610620.0,412904.0,349498.0,294271.0,52979.0
1,ALBANIA,5589.70,4027.54,5227.0,6067.0,9516.0,11452.0,11864.0,15761.0,17712.0,18900.0,25842.0,31305.0,34619.0,41040.0,282091.0,955574.0,10653.0


In [4]:
# Handle missing data with interpolation 

for i in range(len(data)):
    data.iloc[i, 1:] = data.iloc[i, 1:].apply(pd.to_numeric, errors='coerce').interpolate(method='linear')
    
# Fill other Nan values with zero (first column values can't be interpolated)
data.fillna(0, inplace=True)

# Check if there is any Nan values
data.isna().any(axis=1).sum()

0

In [5]:
data.to_csv('cleaned_data.csv', index=False)